## 1. Descargar un dataset de rayos X tipo MNIST

In [1]:
# Para instalar medmnist
!pip install medmnist

In [11]:
from medmnist import INFO
import medmnist

data_flag = 'pneumoniamnist'
info = INFO[data_flag]
DataClass = getattr(medmnist, info['python_class'])  # convierte string en clase

train_dataset = DataClass(split='train', download=True)
test_dataset = DataClass(split='test', download=True)

## 2. Convertirlo a un formato compatible con tu toolkit

El toolkit espera:

    Un CSV con metadatos, donde cada fila contiene al menos: image_id, diagnosis, image_path.

    Una carpeta con imágenes (PNG, JPG o DICOM).

Podemos exportar las imágenes PNG y un CSV con sus etiquetas.

## 3. Código para preparar datos y ejecutar pipeline

In [ ]:
import os
import pandas as pd
from PIL import Image
from medmnist import INFO
import medmnist
import numpy as np

# Configuración
output_data_dir = 'data/mnist_medical_images'
os.makedirs(output_data_dir, exist_ok=True)

# Descargar dataset
data_flag = 'pneumoniamnist'
info = INFO[data_flag]
DataClass = getattr(medmnist, info['python_class'])  # ✅ obtener clase real

dataset = DataClass(split='train', download=True)

# Crear CSV de metadatos
rows = []

for i in range(len(dataset)):
    img_array, label = dataset[i]

    # Asegurar que sea formato PIL.Image
    if isinstance(img_array, np.ndarray):
        img = Image.fromarray(img_array.squeeze())
    else:
        img = img_array

    image_id = f"{i:05d}"
    img_path = os.path.join(output_data_dir, f"{image_id}.png")
    img.save(img_path)

    diagnosis = 'normal' if label == 0 else 'pneumonia'
    rows.append({'image_id': image_id, 'diagnosis': diagnosis, 'filename': img_path.replace(output_data_dir,"")})

metadata_df = pd.DataFrame(rows)
metadata_csv_path = 'data/mnist_medical_metadata.csv'
metadata_df.to_csv(metadata_csv_path, index=False)

print(f"[✅] Metadata CSV saved to {metadata_csv_path}")
print(f"[✅] Images saved to {output_data_dir}")

[✅] Metadata CSV saved to data/mnist_medical_metadata.csv
[✅] Images saved to data/mnist_medical_images


4. Ajusta tu config_default.yaml para que use estos datos:

general:

  * input_metadata_path: "data/
  * mnist_medical_metadata.csv"
  * input_images_dir: "data/
  * mnist_medical_images/"
  * output_dir: "outputs/mnist_experiment"


## 5. Ejecuta el pipeline:

In [ ]:
!python medical_analysis_pipeline.py --config-file config_default.yaml
#!python cli.py run --help

....................
Running step: data_loader


c:\Users\camar\OneDrive\Documents\Analisis-db-medicas\src\steps\data_loader.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
┌───────────────────── Traceback (most recent call last) ─────────────────────┐
│ c:\Users\camar\OneDrive\Documents\Analisis-db-medicas\cli_1.py:22 in run    │
│                                                                             │
│   19 │   │   config['general']['output_dir'] = output_dir                   │
│   20 │                            